# origin
> 기시점, 원점  
>> 참조 point   
>> reference point: one point를 중심으로 모든 대상을 상대적인 비율(거리)로 나타내려할때 그 중심 point 

# axis
> 축
>> 원점에 서있을때 정면 옆면 윗면방향으로 무한히 확장한 축

# perspective
> 관점.  
>> 목적 대상(object)을 원점과의 상대적인 물리적인 관계로 상태(states)를 기술하는 방법

# Scaling Factor
> 축척

>> line
>>> 1D

>> Area
>>> Determinant = scaling factor for areas in matrix

>> Volumn
>>> corss product = scaling factor fo rarea in matix

In [9]:
# 직각좌표계
import sympy as sm
import sympy.vector

### nabla operator = Del()
###    sympy.vector.Del()
nabla = sm.vector.Del()

# 직각 좌표계
B = sm.vector.CoordSys3D('')

# 직각 좌표계의 base vector(i,j,k), base scalar(x,y,z)
B.base_scalars(), B.base_vectors()

# vector in i,j,k vector
v1 = 1*B.i+2*B.j+3*B.k
type(v1)

v2 = 3*B.i+4*B.j+5*B.k
type(v2)

# scalar in x,y,z not a vector
v0 = B.x + B.y + B.z
type(v0)

# to matrix of the Coordinate system,where, v is vector
v1.to_matrix(B)

v1.magnitude() == sm.sqrt(v1.dot(v1))
v1.cross(v2)
v1.normalize()

### projection vector ###
# v1.projection(v2)
# v1.dot(v2) = ||v1|| ||v2|| cos(theta)
# cos(theta) = (v1.dot(v2))/(||v1|| ||v2||)
# v1.project(v2) = ||v2|| cost(theta) \hat{v1} = (v1.dot(v2))/(|\v1||^2) x \vec{v1}
# v1.project(v2) = \frac{||v2||cost(theta)}{\hat{v1}} = \frac{(v1.dot(v2))}{|\v1||^2} \vec{v1}
# v1.project(v2) = \frac{(v1.dot(v2))}{v1.dot(v1)}\vec{v1}
v1.projection(v2) == v1.dot(v2)/v1.dot(v1)*v1

#### nabla Del(), where in field potential energy ###
# \nabla \cdot = divergence
# \nabla \times = curl
# \nabla * f(x,y,z) = gradient, where f(x,y,z) is scalar function(that return real numbers)
u_field = 2 * B.x * B.y**2 * B.i + 3 * B.z**2 * B.y*B.j + 4 * B.x**3 * B.z * B.k
type(u_field)

f_field = 3* B.x**2 + B.y**3 + B.z**2
type(f_field)

nabla = sm.vector.Del()
nabla.dot(u_field)
nabla.dot(u_field).doit() == sm.vector.divergence(u_field)
nabla.cross(u_field)
nabla.cross(u_field).doit() == sm.vector.curl(u_field)
nabla(f_field)
nabla(f_field).doit() == sm.vector.gradient(f_field)

True

In [10]:
#### cylindrical CoordSys3D ####
#### 원통 좌표계
B = sm.vector.CoordSys3D('B')
C = B.create_new('C', transformation='cylindrical')

#### (r,theta,z) -> (x,y.z)
# (r x cos(theta), r x sin(theta), z)
#rv = C.transformation_from_parent()
ra = [ C.r * sm.cos(C.theta), C.r* sm.sin(C.theta), C.z]
rm = sm.Matrix(ra)
rv = sm.vector.matrix_to_vector(rm,C)

#### (r,theta,z) -> (sqrt(x^2 + y^2), atan(y\x), z)
#r_v = C.transformation_from_parent()
####  r > 0, 0 < theta <  2{\pi} -oo < z < oo
r_a = [ sm.sqrt(B.x**2 + B.y**2), sm.atan(B.y/B.x), B.z]
r_m = sm.Matrix(r_a)
r_v = sm.vector.matrix_to_vector(r_m, C)

#### Scale factor Jacobian(행열식) ####
hr = sm.sqrt(rv.diff(C.r).dot(rv.diff(C.r))).simplify()
ht = sm.sqrt(rv.diff(C.theta).dot(rv.diff(C.theta))).simplify()
hz = sm.sqrt(rv.diff(C.z).dot(rv.diff(C.z))).simplify()
#### bases
rh = rv.diff(C.r)/hr
th = rv.diff(C.theta)/ht
zh = rv.diff(C.z)/hz
rh.cross(th).simplify() == zh

#### dS = d{\theta} * dz + 2*dr
## dS = ht * hz * d{\theta}*dz + 2(hr*dr)
## dS = r*d{\theta}*dz + 2(dr)
sm.Integral(C.r,(C.theta,0,2*sm.pi),(C.z,0,C.z)) + 2*sm.Integral(C.r,(C.theta,0,2*sm.pi)) 
sm.integrate(C.r,(C.theta,0,2*sm.pi),(C.z,0,C.z)) + 2*sm.integrate(C.r,(C.theta,0,2*sm.pi)) 
sm.integrate(ht*hz,(C.theta,0,2*sm.pi),(C.z,0,C.z)) + 2*sm.integrate(ht,(C.theta,0,2*sm.pi)) 

#### dV
## dV = hr * ht * hz * dr * dt * dz = dr * r*dt * dz
## dV = dr * r*d{\theta} * dz
sm.Integral(hr*ht*hz,(C.z,0,C.z),(C.theta,0,2*sm.pi),(C.r,0,C.r))
sm.integrate(hr*ht*hz,(C.z,0,C.z),(C.theta,0,2*sm.pi),(C.r,0,C.r))
sm.integrate(C.r,(C.z,0,C.z),(C.theta,0,2*sm.pi),(C.r,0,C.r))

sm.Matrix(C.transformation_to_parent()).subs({C.i:sm.cos(C.theta),C.j:sm.sin(C.theta)})
uf = C.r*sm.cos(C.theta)**2*C.i + C.r*sm.sin(C.theta)**2*C.j + C.z*C.k
type(uf)
uf.to_matrix(C)

sf = C.r**2
type(sf)

nabla = sm.vector.Del()

nabla(sf).doit() == sm.vector.gradient(sf)
nabla.dot(uf).doit() == sm.vector.divergence(uf)
nabla.cross(uf).doit() == sm.vector.curl(uf)
nabla.cross(uf).doit()


((2*C.r*sin(C.theta)**2 + 2*C.r*sin(C.theta)*cos(C.theta))/C.r)*C.k

In [11]:
#### spherical CoordSys3D #####
#### 구 좌표계 ####
B = sm.vector.CoordSys3D('B')
S = B.create_new('S', transformation='spherical')
u = S.r*sm.sin(S.theta)*sm.cos(S.phi)*S.i + S.r*sm.sin(S.theta)*sm.sin(S.phi)*S.j + S.r*sm.cos(S.theta)
u.to_matrix(S)
sm.vector.matrix_to_vector(sm.Matrix(S.transformation_to_parent()),S)

### \ver{r} (r,theta,phi) -> (x,y,z)
ra = [S.r*sm.sin(S.theta)*sm.cos(S.phi), S.r*sm.sin(S.theta)*sm.sin(S.phi),S.r*sm.cos(S.theta)]
rm = sm.Matrix(ra)
rv = sm.vector.matrix_to_vector(rm,S)

#### (x,y,z) ->  (r, theta, phi)
r_a = [sm.sqrt(B.x**2 + B.y**2 + B.z**2),
       sm.atan(sm.sqrt(B.x**2 + B.y**2)/B.z),
       sm.atan(B.y/B.x)]
r_m = sm.Matrix(r_a)
r_x = sm.vector.matrix_to_vector(r_m,S)
#print(f'{r_x}\n{sm.vector.matrix_to_vector(sm.Matrix(S.transformation_from_parent()),S)}')   

### height : length r theta phi 
## 
#hr = rv.diff(S.r).magnitude().doit().simplify()
hr = rv.diff(S.r)
print(f'{hr}')
hr = sm.sqrt(hr.dot(hr)).simplify()
print(f'dr \n{hr} \n{rv.diff(S.r)}')
ht = rv.diff(S.theta)
ht = sm.sqrt(ht.dot(ht)).simplify()

hp = rv.diff(S.phi)
hp = sm.sqrt(hp.dot(hp)).simplify()

rh = rv.diff(S.r) / hr
th = rv.diff(S.theta) / ht
ph = rv.diff(S.phi) / hp

rh.dot(th).doit().simplify()
rh.dot(ph).doit().simplify()
th.dot(ph).doit().simplify()

rh.cross(th).simplify()
th.cross(ph).simplify()

### dS ###
## dS = d{\theta } * d{\phi}
## dr/{\partial \theta}.magnitude() => r * d{\theta}
#print(f'D_theta= {ht.simplify()}')
## dr/{\partial \phi}.magnitude() /phi 축 방향 크기 => r * sin(\theta) d{\phi}
#print(f'D_phi = {hp.simplify()}')
# r^2 x sin({\theta})
# dS = ht x hp x dt x dp = r^2 x sin(theta) x d{\theta} x d{\phi}
sm.Integral(S.r**2 * sm.sin(S.theta),(S.theta,0,sm.pi),(S.phi,0,2*sm.pi))
sm.integrate(S.r**2 * sm.sin(S.theta),(S.theta,0,sm.pi),(S.phi,0,2*sm.pi))

### dV ###
## dV = dr x d{\theta} x d{\phi}
## dr = 1
#print(f'D_r = {hr.simplify()}')
rv.diff(S.r)
sm.Integral(hr*ht*hp,(S.r,0,2*sm.pi),(S.theta,0,sm.pi),(S.phi,0,2*sm.pi))
sm.integrate(hr*ht*hp,(S.r,0,S.r),(S.theta,0,sm.pi),(S.phi,0,2*sm.pi))

### gradient = \nabla(f) ###
nabla = sm.vector.Del()
sm.integrate(S.r**2 * sm.sin(S.theta),(S.theta,0,sm.pi),(S.phi,0,2*sm.pi)) ==  sm.integrate(ht*hp,(S.theta,0,sm.pi),(S.phi,0,2*sm.pi))

(sin(S.theta)*cos(S.phi))*S.i + (sin(S.phi)*sin(S.theta))*S.j + (cos(S.theta))*S.k
dr 
1 
(sin(S.theta)*cos(S.phi))*S.i + (sin(S.phi)*sin(S.theta))*S.j + (cos(S.theta))*S.k


True

# Creating New System v.to_matrix(B)
> ## sympy.vector.CoordSys3D('name')

# Transformaing New System
> ## obj.create_new (name, transforamtion='')
>> ## 원점은 변하지 않는다, axis 만 각과 섞어서 쓴다.
>>> name: str,  
>>> transformation: lambda,tuple.str('cylindrical','spherical'),  
>>> vector_names = ' ',  
>>> variable_names = ' '

In [12]:
C = B.create_new('C','cylindrical')
C = B.create_new(name='C',transformation='cylindrical')
D = B.create_new('D',transformation=lambda x,y,z:(sm.cos(x),sm.sin(y),z))

C.base_vectors()
C.base_scalars()

sm.Matrix(C.transformation_from_parent())
sm.Matrix(C.transformation_to_parent()) 
# wrt = with respect to
#C.position_wrt(B)
sm.vector.express(p,C)

NameError: name 'p' is not defined

In [ ]:
x = sm.symbols('x')
expr = abs(sm.sin(x**2))
sm.ccode(expr)
sm.julia_code(expr)
sm.octave_code(expr)
#import sympy.printing.cxxcode
sm.cxxcode(expr)
sm.ccode?

In [ ]:
S = B.create_new('S','spherical')
S.to_matrix()

# Locating New System
> # obj.locate_new(...)
>> name,  
>> postition,  
>> vector_names,  
>> variable_names


In [ ]:
L = B.locate_new('L', 5*B.i+ 2*B.j + 3*B.k)
sm.Matrix(L.transformation_to_parent()).subs({L.x:7,L.y:2,L.z:3})
sm.Matrix(L.transformation_to_parent())
L.position_wrt(B)
sm.vector.express(p,L)

# Orienting New System
> ## 한축을 중심으로 나머지 origin 회전시킨다..  

> # obj.orient_new(..)
> ## 축의 각도가 변한다.
>> ### obj.orient_new_axis
>>> ### B.orient_new_axis(
        name, 
        angle, 
        axis vector, 
        locastion=$\vec{v}$
)
>> ### obj.orient_new_body
>> ### obj.orient_new_space
>> ### obj.orient_new_quaternion

# QuaternionOrienter
## obj.orient_new(name,(,))v


In [ ]:
theta = sm.symbols('theta')

# \hat{k}를 축으로 해서 \theta 만큼 회전시킨다.
N = B.orient_new_axis('N', theta, B.k)

sm.Matrix(N.transformation_to_parent())
#sm.Matrix(N.transformation_to_parent()).subs({theta:sm.pi/4,N.x:1,N.y:2,N.z:2})
# sm.vector.express(p,N)

In [ ]:
import sympy.functions
N.rotation_matrix(B)
N.rotation_matrix(B).subs({theta:sm.pi/4})
N.rotation_matrix(B).subs({theta:sm.pi/4}).dot([1,2,2])
#sm.Matrix(N.rotation_matrix(B).subs({theta:sm.pi/4}).dot([1,2,2]))

# Orienting and Locating New Coordinate System
> ## B.orient_new_axis (
>> ### 'name',
>> ### $(\angle)$ angle scalar, $(\vec{axis})$ axis vector,
>> ### location = $(\vec{move})$ vector 
> ## )

In [ ]:
a,b,r = sm.symbols('alpha beta gamma')
M = B.orient_new_axis('M',theta,B.k,location= a*B.i + b*B.j + r*B.k)
sm.Matrix(M.transformation_to_parent()).subs({theta:sm.pi/4,a:1,b:1,r:1,M.x:1,M.y:2,M.z:2})

# quaternion
> ## sympy.algebras.quaternion
>> ## $q = (w , \vec{v})$
>>> ## $q = (w,(x,y,z))$
>>> ## $q = w + xi + yj + zk \\ q_1 = w_1 + \vec{v_1} \\ q_2= w_2 + \vec{v_2}$
> ## $q_1\:q_2 = (w_1w_2 - \vec{v_1}\cdot \vec{v_2}, \quad w_1\vec{v_1} + w_2\vec{v_2} + \vec{v_1}\times\vec{v_2})$
> ## $q^2 = (0,\vec{v})(0,\vec{v}) = (-\vec{v}\cdot \vec{v}, \vec{0}) = -|\vec{v}|^2 $

In [ ]:
import sympy.algebras
x = sm.symbols('x')
q = sm.algebras.Quaternion(1,2,3,4)
q1 = sm.algebras.Quaternion(x,x**3,x,x**2,real_field=False)
q2 = sm.algebras.Quaternion(3+4*sm.I,2+5*sm.I,0,7+8*sm.I,real_field=False)
q2

# 
> ### $ \vec{v} = ai + bj + ck \\
q = w + \vec{v} \\
\quad = (w, \vec{v}) \\
\quad = (w, ai + bj + ck) $
>> ### $ e^{ai + bj + ck} = cos(|\vec{v}|) + 
\frac{sin(|\vec{v}|}{|\vec{v}|} 
(ai + bj + ck)$  
>> ### $ e^{w + ai + bj + ck} \\
\quad = e^{w} e^{ai+bj+ck} \\
\quad = cos(|\vec{v}|) +  \frac{sin(|\vec{v}|}{|\vec{v}|} 
(ai + bj + ck) \\
\quad = cos(|\vec{v}|) + sin(|\vec{v}|)\,
\frac {ai + bj + ck}{|\vec{v}|} \\
\quad = cos(|\vec{v}|) + sin(|\vec{v}|)\,
\frac {\vec{v}}{|\vec{v}|} \\
\quad = cos(|\vec{v}|) + sin(|\vec{v}|)\,\hat{v}\\
$
> ## $ e^{\vec{v}} = e^{ai+bj+ck} 
\begin{cases}
cos(|\vec{v}|) +  
\frac{sin(|\vec{v}|}{|\vec{v}|}\,(ai + bj + ck) \\
cos(|\vec{v}|) + 
sin(|\vec{v}|)\,
(
\frac{ai}{|\vec{v}|} + 
\frac{bj}{|\vec{v}|} + 
\frac{ck}{|\vec{v}|}
) \\
cos(|\vec{v}|) + sin(|\vec{v}|)\,\hat{v} \\
\end{cases}$

In [ ]:
# e^q
q = sm.algebras.Quaternion(sm.cos(theta),
                           sm.sin(theta)*sm.sqrt(1/3),
                           sm.sin(theta)*sm.sqrt(1/3),
                           sm.sin(theta)*sm.sqrt(1/3))
q.mul(q).trigsimp()

In [ ]:
sm.exp(sm.algebras.Quaternion(1,1,1,1)) 

In [ ]:
sm.algebras.Quaternion(1,1,1,1).exp()

# Quaternion Multiply
> #[quaternion](https://www.youtube.com/watch?v=88BA8aO3qXA)
> $ (a + bi + cj + dk)\:(e + fi + gj + hk ) \\
\begin{bmatrix}
a & -b & -c & -d \\
b &  a & -d & c \\
c &  d &  a & -b \\
d & -c &  b & a
\end{bmatrix} \quad
\begin{bmatrix}
e \\ f \\ g \\ h 
\end{bmatrix}
$
> # unit quaternion matrix
>> $ 1 = (1,0,0,0) \\
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 
\end{bmatrix} 
$  
>> $ i = (0,1,0,0) \\
\begin{bmatrix}
0 & -1 & 0 & 0  \\
1 & 0 & 0 & 0 \\
0 & 0 & 0 & -1 \\
0 & 0 & 1 & 0 \\
\end{bmatrix}
$  
>> $ j = (0,0,1,0) \\
\begin{bmatrix}
0 & 0 & -1 & 0 \\
0 & 0 & 0 & -1 \\
1 & 0 & 0 & 0 \\
0 & -1 & 0 & 0 \\
\end{bmatrix}
$  
>> $k = (0,0,0,1) \\
\begin{bmatrix}
0 & 0 & 0 & -1 \\
0 & 0 & -1 & 0 \\
0 & 1 & 0 & 0 \\
1 & 0 & 0 & 0 \\
\end{bmatrix}
$

In [ ]:
q0, q1, q2, q3 = sm.symbols('q_0 q_1 q_2 q_3')
Q = B.orient_new_quaternion('Q',q0,q1,q2,q3)
sm.Matrix(Q.transformation_to_parent())

In [ ]:
m = sm.Matrix([1,2,3])
q = sm.algebras.Quaternion(1,2,3,4)

q.norm()
q.normalize()
q.inverse()
q.pow(2) == q.mul(q) == q*q
sm.vector.divergence(q)
sm.vector.curl(q)
sm.vector.gradient(q)

In [24]:
x = sm.symbols('x')
q1 = sm.algebras.Quaternion(x**2,x**3,x)
q2 = sm.algebras.Quaternion(2,(3+2*sm.I), x**2, 3.5*sm.I)

In [25]:
q1 * q2

(-x**3 - x**3*(3 + 2*I) + 2*x**2) + (2*x**3 + x**2*(3 + 2*I) + 3.5*I*x)*i + (x**4 - 3.5*I*x**3 + 2*x)*j + (x**5 + 3.5*I*x**2 - x*(3 + 2*I))*k

In [30]:
q1.inverse()
q1.conjugate()/q1.norm()

x**2/sqrt(x**6 + x**4 + x**2) + (-x**3/sqrt(x**6 + x**4 + x**2))*i + (-x/sqrt(x**6 + x**4 + x**2))*j + 0*k

In [32]:
q1.inverse()

x**2/(x**6 + x**4 + x**2) + (-x**3/(x**6 + x**4 + x**2))*i + (-x/(x**6 + x**4 + x**2))*j + 0*k